In [1]:
import sys
import os
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

In [2]:
import src
from src.exceptions import *
from src.cvar_opt import *
from src.returns_sim import *

In [3]:
!pip show yfinance
!pip show py_vollib

Name: yfinance
Version: 0.2.38
Summary: Download market data from Yahoo! Finance API
Home-page: https://github.com/ranaroussi/yfinance
Author: Ran Aroussi
Author-email: ran@aroussi.com
License: Apache
Location: /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages
Requires: appdirs, beautifulsoup4, frozendict, html5lib, lxml, multitasking, numpy, pandas, peewee, pytz, requests
Required-by: 
Name: py-vollib
Version: 1.0.1
Summary: UNKNOWN
Home-page: http://vollib.org
Author: 
Author-email: 
License: MIT
Location: /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages
Requires: numpy, pandas, py-lets-be-rational, scipy, simplejson
Required-by: 


In [4]:
import yfinance as yf
import numpy as np
import pandas as pd

from py_vollib.black_scholes_merton.greeks.analytical import delta, gamma, vega, theta, rho

We will analyze SPY data, an ETF that tracks the S&P 500 index. Options data were downloaded from CBOE the night of April 22, 2024. Thus, data with the closing price of the SPY is utilized for the 

In [5]:
spy = yf.Ticker("SPY")
spy_hist = spy.history(period="1mo")
current_price = spy_hist.loc["2024-04-22 00:00:00-04:00", "Close"]
current_price

499.7200012207031

CBOE Options data the night of April 22, 2024, downloaded as a CSV file from https://www.cboe.com/delayed_quotes/spy/quote_table

In [6]:
spy_data_path = os.getcwd() + "/spy_quotedata.csv"
spy_data = pd.read_csv(spy_data_path, skiprows=3)
spy_one_month_options_exp = spy_data[spy_data["Expiration Date"] == "Fri May 24 2024"]
spy_one_month_options_exp

,Expiration Date,Calls,Last Sale,Net,Bid,Ask,Volume,IV,Delta,Gamma,...,Puts,Last Sale.1,Net.1,Bid.1,Ask.1,Volume.1,IV.1,Delta.1,Gamma.1,Open Interest.1
487,Fri May 24 2024,SPY240524C00395000,102.59,0.000,106.46,106.93,0,0.3443,0.9904,0.0005,...,SPY240524P00395000,0.18,-0.155,0.18,0.19,17,0.3572,-0.0105,0.0005,65
488,Fri May 24 2024,SPY240524C00400000,107.68,0.000,101.51,101.97,0,0.3342,0.9892,0.0006,...,SPY240524P00400000,0.19,-0.180,0.19,0.21,6,0.3440,-0.0117,0.0006,120
489,Fri May 24 2024,SPY240524C00405000,94.47,0.000,96.56,97.02,0,0.3244,0.9879,0.0007,...,SPY240524P00405000,0.22,-0.190,0.21,0.23,80,0.3317,-0.0131,0.0007,12125
490,Fri May 24 2024,SPY240524C00410000,92.50,0.000,91.61,92.07,0,0.3137,0.9862,0.0008,...,SPY240524P00410000,0.25,-0.205,0.24,0.25,33,0.3200,-0.0147,0.0008,934
491,Fri May 24 2024,SPY240524C00415000,84.64,0.000,86.66,87.12,0,0.3021,0.9843,0.0009,...,SPY240524P00415000,0.32,-0.190,0.26,0.28,5,0.3075,-0.0167,0.0009,301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,Fri May 24 2024,SPY240524C00580000,0.02,0.005,0.01,0.02,101,0.1695,0.0022,0.0003,...,SPY240524P00580000,0.00,0.000,80.50,80.96,0,0.0000,-1.0000,0.0000,0
573,Fri May 24 2024,SPY240524C00585000,0.01,0.000,0.01,0.02,0,0.1785,0.0018,0.0002,...,SPY240524P00585000,0.00,0.000,85.50,85.96,0,0.0000,-1.0000,0.0000,0
574,Fri May 24 2024,SPY240524C00590000,0.01,-0.005,0.01,0.02,3,0.1875,0.0015,0.0002,...,SPY240524P00590000,0.00,0.000,90.49,90.96,0,0.0000,-1.0000,0.0000,0
575,Fri May 24 2024,SPY240524C00595000,0.03,0.000,0.01,0.02,0,0.1963,0.0012,0.0001,...,SPY240524P00595000,0.00,0.000,95.49,95.96,0,0.0000,-1.0000,0.0000,0


In [7]:
puts_index = spy_one_month_options_exp.columns.get_loc('Puts')
calls_data = spy_one_month_options_exp.iloc[:, :puts_index]
puts_data = spy_one_month_options_exp.iloc[:, [0] + list(range(puts_index-1, len(spy_one_month_options_exp.columns)))]
calls_data.head()
puts_data.head()

,Expiration Date,Strike,Puts,Last Sale.1,Net.1,Bid.1,Ask.1,Volume.1,IV.1,Delta.1,Gamma.1,Open Interest.1
487,Fri May 24 2024,395.0,SPY240524P00395000,0.18,-0.155,0.18,0.19,17,0.3572,-0.0105,0.0005,65
488,Fri May 24 2024,400.0,SPY240524P00400000,0.19,-0.180,0.19,0.21,6,0.3440,-0.0117,0.0006,120
489,Fri May 24 2024,405.0,SPY240524P00405000,0.22,-0.190,0.21,0.23,80,0.3317,-0.0131,0.0007,12125
490,Fri May 24 2024,410.0,SPY240524P00410000,0.25,-0.205,0.24,0.25,33,0.3200,-0.0147,0.0008,934
491,Fri May 24 2024,415.0,SPY240524P00415000,0.32,-0.190,0.26,0.28,5,0.3075,-0.0167,0.0009,301


Determine the +/- 30 and 70 delta calls and puts as well as the calls and puts with strike price closest to the current stock price in order to determine the In the Money, Out of the Money, and At the Money calls and puts.

In [8]:
# Calculate the absolute difference between delta and 0.30 for calls_data -- Out of the Money Option
calls_data['Delta Difference'] = abs(calls_data['Delta'] - 0.30)

# Find the row with the smallest difference
otm_call_option = calls_data.loc[calls_data['Delta Difference'].idxmin()]

print("Option with Delta Closest to 0.30:")
print(otm_call_option)

# Calculate the absolute difference between delta and 0.70 for calls_data -- In the Money Option
calls_data['Delta Difference'] = abs(calls_data['Delta'] - 0.70)

# Find the row with the smallest difference
itm_call_option = calls_data.loc[calls_data['Delta Difference'].idxmin()]

print("Option with Delta Closest to 0.70:")
print(itm_call_option)

# Calculate the absolute difference between strike price and current price for calls_data -- At the Money Option
calls_data['Price Difference'] = abs(calls_data['Strike'] - current_price)

# Find the row with the smallest difference
atm_call_option = calls_data.loc[calls_data['Price Difference'].idxmin()]

print("At the Money Option:")
print(atm_call_option)

Option with Delta Closest to 0.30:
Expiration Date        Fri May 24 2024
Calls               SPY240524C00513000
Last Sale                         4.41
Net                              1.005
Bid                               3.92
Ask                               3.97
Volume                             197
IV                              0.1385
Delta                           0.3086
Gamma                           0.0172
Open Interest                      451
Strike                           513.0
Delta Difference                0.0086
Name: 537, dtype: object
Option with Delta Closest to 0.70:
Expiration Date        Fri May 24 2024
Calls               SPY240524C00490000
Last Sale                        16.95
Net                              2.375
Bid                              16.62
Ask                              16.74
Volume                              43
IV                              0.1631
Delta                           0.7017
Gamma                           0.0144
Open Int

In [9]:
# Calculate the absolute difference between delta and -0.30 for puts_data -- Out of the Money Option
puts_data['Delta Difference'] = abs(puts_data['Delta.1'] + 0.30)

# Find the row with the smallest difference
otm_put_option = puts_data.loc[puts_data['Delta Difference'].idxmin()]

print("Option with Delta Closest to -0.30:")
print(otm_put_option)

# Calculate the absolute difference between delta and 0.70 for puts_data -- In the Money Option
puts_data['Delta Difference'] = abs(puts_data['Delta.1'] + 0.70)

# Find the row with the smallest difference
itm_put_option = puts_data.loc[puts_data['Delta Difference'].idxmin()]

print("Option with Delta Closest to -0.70:")
print(itm_put_option)

# Calculate the absolute difference between strike price and current price for puts_data -- At the Money Option
puts_data['Price Difference'] = abs(puts_data['Strike'] - current_price)

# Find the row with the smallest difference
atm_put_option = puts_data.loc[puts_data['Price Difference'].idxmin()]

print("At the Money Option:")
print(atm_put_option)

Option with Delta Closest to -0.30:
Expiration Date        Fri May 24 2024
Strike                           489.0
Puts                SPY240524P00489000
Last Sale.1                       4.56
Net.1                           -2.605
Bid.1                             4.48
Ask.1                             4.54
Volume.1                           590
IV.1                            0.1635
Delta.1                        -0.2943
Gamma.1                         0.0146
Open Interest.1                   2145
Delta Difference                0.0057
Name: 513, dtype: object
Option with Delta Closest to -0.70:
Expiration Date        Fri May 24 2024
Strike                           511.0
Puts                SPY240524P00511000
Last Sale.1                      13.58
Net.1                           -4.855
Bid.1                            13.97
Ask.1                            14.27
Volume.1                            32
IV.1                            0.1398
Delta.1                        -0.7005
Gamma.

/var/folders/h4/3615n54s4mb6l7t66t_wc5540000gn/T/ipykernel_16418/4070944909.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  puts_data['Delta Difference'] = abs(puts_data['Delta.1'] + 0.30)
/var/folders/h4/3615n54s4mb6l7t66t_wc5540000gn/T/ipykernel_16418/4070944909.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  puts_data['Delta Difference'] = abs(puts_data['Delta.1'] + 0.70)
/var/folders/h4/3615n54s4mb6l7t66t_wc5540000gn/T/ipykernel_16418/4070944909.py:20: SettingWithCopyWarning: 
A value is trying

In [10]:
# Create the Options Dictionary to Simulate Returns
options_dict = {}

itm_call_option_data = {
    "Expiration Date": itm_call_option["Expiration Date"],
    "Instrument Name": itm_call_option["Calls"],
    "Strike": itm_call_option["Strike"],
    "Price": (itm_call_option["Bid"] + itm_call_option["Ask"]) / 2,
    "IV": itm_call_option["IV"],
    "Delta": itm_call_option["Delta"],
    "Gamma": itm_call_option["Gamma"]
}
otm_call_option_data = {
    "Expiration Date": otm_call_option["Expiration Date"],
    "Instrument Name": otm_call_option["Calls"],
    "Strike": otm_call_option["Strike"],
    "Price": (otm_call_option["Bid"] + otm_call_option["Ask"]) / 2,
    "IV": otm_call_option["IV"],
    "Delta": otm_call_option["Delta"],
    "Gamma": otm_call_option["Gamma"]
}
atm_call_option_data = {
    "Expiration Date": atm_call_option["Expiration Date"],
    "Instrument Name": atm_call_option["Calls"],
    "Strike": atm_call_option["Strike"],
    "Price": (atm_call_option["Bid"] + atm_call_option["Ask"]) / 2,
    "IV": atm_call_option["IV"],
    "Delta": atm_call_option["Delta"],
    "Gamma": atm_call_option["Gamma"]
}
itm_put_option_data = {
    "Expiration Date": itm_put_option["Expiration Date"],
    "Instrument Name": itm_put_option["Puts"],
    "Strike": itm_put_option["Strike"],
    "Price": (itm_put_option["Bid.1"] + itm_put_option["Ask.1"]) / 2,
    "IV": itm_put_option["IV.1"],
    "Delta": itm_put_option["Delta.1"],
    "Gamma": itm_put_option["Gamma.1"]
}
atm_put_option_data = {
    "Expiration Date": atm_put_option["Expiration Date"],
    "Instrument Name": atm_put_option["Puts"],
    "Strike": atm_put_option["Strike"],
    "Price": (atm_put_option["Bid.1"] + atm_put_option["Ask.1"]) / 2,
    "IV": atm_put_option["IV.1"],
    "Delta": atm_put_option["Delta.1"],
    "Gamma": atm_put_option["Gamma.1"]
}
otm_put_option_data = {
    "Expiration Date": otm_put_option["Expiration Date"],
    "Instrument Name": otm_put_option["Puts"],
    "Strike": otm_put_option["Strike"],
    "Price": (otm_put_option["Bid.1"] + otm_put_option["Ask.1"]) / 2,
    "IV": otm_put_option["IV.1"],
    "Delta": otm_put_option["Delta.1"],
    "Gamma": otm_put_option["Gamma.1"]
}

options_dict["ITM Call Option"] = itm_call_option_data
options_dict["ATM Call Option"] = atm_call_option_data
options_dict["OTM Call Option"] = otm_call_option_data
options_dict["ITM Put Option"] = itm_put_option_data
options_dict["ATM Put Option"] = atm_put_option_data
options_dict["OTM Put Option"] = otm_put_option_data

# Convert the dictionary to a DataFrame
options_df = pd.DataFrame.from_dict(options_dict, orient='index')

The US 10 year is the risk free rate, $r$, the spy yield is the dividend parameter, $q$, and the days to expiry is counted out of 360 days, and will be scaled down to trading days by a factor of $\frac{252}{360}$. These parameters are used for calculation of the Options Greeks with the Black Scholes Merton model.

In [11]:
us_10y = yf.Ticker("^TNX").history("1mo").loc["2024-04-22 00:00:00-05:00", "Close"] / 100
spy_yield = spy.info["yield"]
dte = 31

The options_df Dataframe contains different approximately 1 month to expiry options, with their respective greeks, and implied volatility which are used to determine simulations and returns.

In [12]:
# Use of Black-Scholes Model to compute theta values
theta_itm_call = theta('c', current_price, options_df.loc["ITM Call Option", "Strike"], dte/360, us_10y, options_df.loc["ITM Call Option", "IV"], spy_yield)
theta_atm_call = theta('c', current_price, options_df.loc["ATM Call Option", "Strike"], dte/360, us_10y, options_df.loc["ATM Call Option", "IV"], spy_yield)
theta_otm_call = theta('c', current_price, options_df.loc["OTM Call Option", "Strike"], dte/360, us_10y, options_df.loc["OTM Call Option", "IV"], spy_yield)
theta_itm_put = theta('p', current_price, options_df.loc["ITM Put Option", "Strike"], dte/360, us_10y, options_df.loc["ITM Put Option", "IV"], spy_yield)
theta_atm_put = theta('p', current_price, options_df.loc["ATM Put Option", "Strike"], dte/360, us_10y, options_df.loc["ATM Put Option", "IV"], spy_yield)
theta_otm_put = theta('p', current_price, options_df.loc["OTM Put Option", "Strike"], dte/360, us_10y, options_df.loc["OTM Put Option", "IV"], spy_yield)

options_df.loc["ITM Call Option", "Theta"] = theta_itm_call
options_df.loc["ATM Call Option", "Theta"] = theta_atm_call
options_df.loc["OTM Call Option", "Theta"] = theta_otm_call
options_df.loc["ITM Put Option", "Theta"] = theta_itm_put
options_df.loc["ATM Put Option", "Theta"] = theta_atm_put
options_df.loc["OTM Put Option", "Theta"] = theta_otm_put

options_df

,Expiration Date,Instrument Name,Strike,Price,IV,Delta,Gamma,Theta
ITM Call Option,Fri May 24 2024,SPY240524C00490000,490.0,16.680,0.1631,0.7017,0.0144,-0.163669
ATM Call Option,Fri May 24 2024,SPY240524C00500000,500.0,9.960,0.1509,0.5464,0.0177,-0.162918
OTM Call Option,Fri May 24 2024,SPY240524C00513000,513.0,3.945,0.1385,0.3086,0.0172,-0.123213
ITM Put Option,Fri May 24 2024,SPY240524P00511000,511.0,14.120,0.1398,-0.7005,0.0203,-0.084630
ATM Put Option,Fri May 24 2024,SPY240524P00500000,500.0,8.140,0.1508,-0.4740,0.0190,-0.117254
OTM Put Option,Fri May 24 2024,SPY240524P00489000,489.0,4.510,0.1635,-0.2943,0.0146,-0.117543


In [13]:
simulated_returns, mean_returns, stdev_returns = sim_returns_from_options_data(current_price, options_df, 31, 1000000, "SPY")
simulated_returns.head()

,SPY,ITM Call Option,ATM Call Option,OTM Call Option,ITM Put Option,ATM Put Option,OTM Put Option
0,-0.000972,-0.030355,-0.043223,-0.069757,0.018282,0.014165,0.006027
1,0.059615,1.626527,2.406533,4.233879,-2.121892,-2.784921,-3.406577
2,-0.027663,-0.673830,-0.944501,-1.529157,0.817161,1.013568,1.185293
3,-0.017598,-0.413148,-0.567513,-0.887747,0.485878,0.587941,0.672974
4,0.007981,0.164830,0.216566,0.315420,-0.215284,-0.265204,-0.312058


In [14]:
mean_returns

SPY                0.000573
ITM Call Option    0.008329
ATM Call Option    0.011904
OTM Call Option    0.021975
ITM Put Option    -0.030353
ATM Put Option    -0.047569
OTM Put Option    -0.067622
dtype: float64

In [15]:
stdev_returns

SPY                0.032416
ITM Call Option    0.866275
ATM Call Option    1.271985
OTM Call Option    2.217754
ITM Put Option     1.113126
ATM Put Option     1.448790
OTM Put Option     1.761166
dtype: float64

Utilization of our CVAR_Opt library to create a portfolio that minimizes CVaR, using implied volatility to determine expected move. Simulated Returns assume a delta-gamma Taylor Series approximation of return, and does not incorporated jumps (as with a Merton Jump Diffusion options pricing model), nor does it consider option pricing changes due to a change in implied volatility, and thus vega, nor pricing changes due to a change in the risk free rate, and thus rho.

In [16]:
# CVaR Minimization
ec = CVaROpt(mean_returns, simulated_returns, 1000000, verbose=True)

In [17]:
portfolio_weight = ec.min_cvar()
portfolio_weight 

                                     CVXPY                                     
                                     v1.4.3                                    
(CVXPY) Apr 23 03:37:44 PM: Your problem has 1000008 variables, 5 constraints, and 0 parameters.
(CVXPY) Apr 23 03:37:44 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Apr 23 03:37:44 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Apr 23 03:37:44 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Apr 23 03:37:44 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Apr 23 03:37:44 PM: Compiling problem (target solver=ECOS).

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/cvxpy/reductions/solvers/solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)


(CVXPY) Apr 23 03:37:48 PM: Applying reduction ECOS
(CVXPY) Apr 23 03:37:51 PM: Finished problem compilation (took 6.421e+00 seconds).
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
(CVXPY) Apr 23 03:37:51 PM: Invoking solver ECOS  to obtain a solution.
-------------------------------------------------------------------------------
                                    Summary                                    
-------------------------------------------------------------------------------
(CVXPY) Apr 23 03:40:17 PM: Problem status: optimal_inaccurate
(CVXPY) Apr 23 03:40:17 PM: Optimal value: -3.262e-02
(CVXPY) Apr 23 03:40:17 PM: Compilation took 6.421e+00 seconds
(CVXPY) Apr 23 03:40:17 PM: Solver (including time spent in interface) took 1.461e+02 seconds


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/cvxpy/problems/problem.py:1403: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


OrderedDict([('SPY', 1.0),
             ('ITM Call Option', 1.0),
             ('ATM Call Option', -0.5491867052285975),
             ('OTM Call Option', -0.4091938814538336),
             ('ITM Put Option', 0.9999999999999998),
             ('ATM Put Option', -0.0416194133175688),
             ('OTM Put Option', -1.0)])

In [18]:
ec.portfolio_performance(verbose=True)

Expected daily return: 3.3%
Conditional Value at Risk: -3.26%


(0.032619897053071605, -0.0326198970530698)

The delta, gamma, and theta of the net portfolio that minimizes CVaR. 

In [19]:
# Portfolio Greeks
options_choices = ['ITM Call Option', 'ATM Call Option', 'OTM Call Option', 'ITM Put Option', 'ATM Put Option', 'OTM Put Option']
delta = portfolio_weight['SPY']
theta = 0
gamma = 0
for option in options_choices:
    delta += portfolio_weight[option] * options_df.loc[option, 'Delta']
    theta += portfolio_weight[option] * options_df.loc[option, 'Theta']
    gamma += portfolio_weight[option] * options_df.loc[option, 'Gamma']

print("Net Delta: {:.4f}".format(delta))
print("Net Theta: {:.4f}".format(theta))
print("Net Gamma: {:.4f}".format(gamma))

Net Delta: 0.8889
Net Theta: 0.0140
Net Gamma: 0.0026


Given a required minimum return, minimize CVaR

In [20]:
# 5% Daily Return required
ec = CVaROpt(mean_returns, simulated_returns, 1000000, verbose=True)
portfolio_weight_req_return = ec.min_cvar_with_req_returns(0.05)
portfolio_weight_req_return

                                     CVXPY                                     
                                     v1.4.3                                    
(CVXPY) Apr 23 03:40:17 PM: Your problem has 1000008 variables, 6 constraints, and 1 parameters.
(CVXPY) Apr 23 03:40:17 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Apr 23 03:40:17 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Apr 23 03:40:17 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Apr 23 03:40:17 PM: Compiling problem (target solver=ECOS).
(CVXPY) Apr 23 03:40:17 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> ECOS
(CVXPY) Apr 23 03:40:17 PM: A

63  +9.913e-01  +9.913e-01  +1e-06  2e-14  1e-11  6e-15  5e-13  0.9890  6e-01 -------------------------------------------------------------------------------
                                    Summary                                    
-------------------------------------------------------------------------------
(CVXPY) Apr 23 03:42:50 PM: Problem status: optimal_inaccurate
(CVXPY) Apr 23 03:42:50 PM: Optimal value: 9.913e-01
(CVXPY) Apr 23 03:42:50 PM: Compilation took 5.665e+00 seconds
(CVXPY) Apr 23 03:42:50 PM: Solver (including time spent in interface) took 1.475e+02 seconds


OrderedDict([('SPY', 1.0000000000008096),
             ('ITM Call Option', 1.0000000000000977),
             ('ATM Call Option', 0.4338955553090586),
             ('OTM Call Option', -1.0000000000000615),
             ('ITM Put Option', 1.0000000000001183),
             ('ATM Put Option', -0.4338955553107492),
             ('OTM Put Option', -1.0000000000000875)])

In [21]:
ec.portfolio_performance(verbose=True)

Expected daily return: 5.0%
Conditional Value at Risk: 99.13%


(0.04999999999993904, 0.9913332627610215)

In [22]:
# Portfolio Greeks
options_choices = ['ITM Call Option', 'ATM Call Option', 'OTM Call Option', 'ITM Put Option', 'ATM Put Option', 'OTM Put Option']
delta = portfolio_weight_req_return['SPY']
theta = 0
gamma = 0
for option in options_choices:
    delta += portfolio_weight_req_return[option] * options_df.loc[option, 'Delta']
    theta += portfolio_weight_req_return[option] * options_df.loc[option, 'Theta']
    gamma += portfolio_weight_req_return[option] * options_df.loc[option, 'Gamma']

print("Net Delta: {:.4f}".format(delta))
print("Net Theta: {:.4f}".format(theta))
print("Net Gamma: {:.4f}".format(gamma))

Net Delta: 1.4296
Net Theta: -0.0274
Net Gamma: 0.0023


Given a target CVaR, maximize mean / expected portfolio returns.

In [23]:
# 2% CVaR at most
ec = CVaROpt(mean_returns, simulated_returns, 1000000, verbose=True)
portfolio_weight_req_cvar = ec.max_return_with_req_cvar(0.02)
portfolio_weight_req_cvar

                                     CVXPY                                     
                                     v1.4.3                                    
(CVXPY) Apr 23 03:42:50 PM: Your problem has 1000008 variables, 6 constraints, and 1 parameters.
(CVXPY) Apr 23 03:42:50 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Apr 23 03:42:50 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Apr 23 03:42:50 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Apr 23 03:42:50 PM: Compiling problem (target solver=ECOS).
(CVXPY) Apr 23 03:42:50 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> ECOS
(CVXPY) Apr 23 03:42:50 PM: A

OrderedDict([('SPY', 1.000000000000024),
             ('ITM Call Option', 0.999999999999978),
             ('ATM Call Option', -0.1232724149239402),
             ('OTM Call Option', -0.7202871709933818),
             ('ITM Put Option', 1.0000000000000009),
             ('ATM Put Option', -0.1564404140827086),
             ('OTM Put Option', -1.0000000000000002)])

In [24]:
ec.portfolio_performance(verbose=True)

Expected daily return: 3.6%
Conditional Value at Risk: 2.00%


(0.03631576750763314, 0.020000000000671043)

In [25]:
# Portfolio Greeks
options_choices = ['ITM Call Option', 'ATM Call Option', 'OTM Call Option', 'ITM Put Option', 'ATM Put Option', 'OTM Put Option']
delta = portfolio_weight_req_cvar['SPY']
theta = 0
gamma = 0
for option in options_choices:
    delta += portfolio_weight_req_cvar[option] * options_df.loc[option, 'Delta']
    theta += portfolio_weight_req_cvar[option] * options_df.loc[option, 'Theta']
    gamma += portfolio_weight_req_cvar[option] * options_df.loc[option, 'Gamma']

print("Net Delta: {:.4f}".format(delta))
print("Net Theta: {:.4f}".format(theta))
print("Net Gamma: {:.4f}".format(gamma))

Net Delta: 1.0800
Net Theta: -0.0036
Net Gamma: 0.0026
